# Search Work and Export BibTeX

In [1]:
import os, sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))
import database
from snowballing.operations import load_work, work_to_bibtex, reload
reload()

In [2]:
def find(text):
    words = text.split()
    for work in load_work():
        match = True
        for word in words:
            if not any(word.lower() in str(getattr(work, attr)).lower() for attr in dir(work) if not attr.startswith("_")):
                match = False
                break
        if match:
            yield work_to_bibtex(work)

In [3]:
from ipywidgets import widgets, interactive

def result(text):
    if len(text) > 2:
        for work in find(text):
            print(work.replace("\n ", "\n  "))
interactive(result, text="")

# Match BibTeX file to database work

In [11]:
bibtex_file = '../../csur/bibliography.bib'

from snowballing.operations import match_bibtex_to_work
with open(bibtex_file) as fil:
    bibtex_str = fil.read()

matched = match_bibtex_to_work(bibtex_str.split("%Entries")[-1])
works = dict(map(reversed, matched))

### Check if all snowballed approaches appear in the bibtex

In [12]:
from snowballing.approaches import get_approaches
all_approaches = get_approaches()
script = [(a, m) for a, m in all_approaches if not m["binary"]]
binary = [(a, m) for a, m in all_approaches if m["binary"]]
len(script)

27

In [13]:
for a, m in script:
    for w in a.work:
        if w not in works and "snowball" in w.category:
            print(w.metakey, w)

### Look for unmatched work

In [14]:
from copy import copy
from snowballing.operations import info_to_code, bibtex_to_info
unmatched = [x[0] for x in matched if not x[1]]
for unmatch in unmatched:
    print(info_to_code(bibtex_to_info(copy(unmatch))))

### Recreate Bibtex

In [15]:
from snowballing.operations import work_to_bibtex_entry
result = [
    work_to_bibtex_entry(work, name=entry['ID'], acronym=True)
    for entry, work in matched
]

In [16]:
from bibtexparser.bwriter import BibTexWriter
from bibtexparser.bibdatabase import BibDatabase
db = BibDatabase()
db.entries = result

writer = BibTexWriter()
writer.indent = '  '     # indent entries with 4 spaces instead of one

In [17]:
with open(bibtex_file, 'w') as fil:
    fil.write(bibtex_str.split("%Entries")[0] + "%Entries\n\n" + writer.write(db))